## 10.1 Image Pyramid
- 10.1.1 Gaussian Pyramid
- 10.1.2 Laplacian Pyramid

In [ ]:
import cv2

import numpy as np

___
# <font color="orange">10.1 Image Pyramid</font>

<img src="resource/Pyramids.png" style="width:400px"></img>
- An <font color="orange">image pyramid</font> is a collection of images - all arising from a single original image - that are successively downsampled until some desired stopping point is reached.
- Although there is a *geometric transformation* function in OpenCV that -literally- resize an image (`cv2.resize()`), in this section we will use of Image Pyramids, which are widely applied in a huge range of vision applications.<br><br><br>
- There are two common kinds of image pyramids:
    - <font color="orange">**Gaussian pyramid**</font>: Used to **downsample** & **upsample** images
    - <font color="orange">**Laplacian pyramid**</font>: Used to reconstruct an **upsampled** image from an image lower in the pyramid (with less resolution)<br><br><br><br>
___
## 1.1 <font color="orange">Gaussian Pyramid</font>
- Imagine the pyramid as a set of layers in which the higher the layer, the smaller the size.
- Every layer is numbered from bottom to top, so layer ($i+1$) (denoted as $G_i+1$ is smaller than layer $i ( G_i)$.<br><br>
<img src="resource/Pyramids_Tutorial_Pyramid_Theory.png" style="width:400px"></img><br><br><br>

- To produce layer ($i+1$) in the Gaussian pyramid, we do the following:
    - <font color="orange">**Convolve**</font> $G_i$ with a *Gaussian kernel*:<br>
    $\frac{1}{16} \begin{bmatrix} 1 & 4 & 6 & 4 & 1 \\ 4 & 16 & 24 & 16 & 4 \\ 6 & 24 & 36 & 24 & 6 \\ 4 & 16 & 24 & 16 & 4 \\ 1 & 4 & 6 & 4 & 1 \end{bmatrix}$
    - Remove every even-numbered row and column.   
- The resulting image will be exactly <font color="orange">*one-quarter*</font> the area of its predecessor. Iterating this process on the input image $G_0$ (original image) produces the entire pyramid.
- The procedure above was useful to **downsample** an image. What if we want to make it bigger?
    - First, <font color="orange">**upsize**</font> the image to twice the original in each dimension, with the new even rows and
    - Perform a <font color="orange">**convolution**</font> with the same kernel shown above (multiplied by 4) to approximate the values of the "missing pixels"

- Downsampling and upsampling method in OpenCV is used `cv2.pyrUp(img, dstsize, borderType)` and `cv2.pyrDown(img, dstsize, borderType)`
- where :
    - `img` : input image
    - `dstsize` : image destination size, default ($0.5 w, 0.5 h$) for downscale, ($2w, 2h$) for upscale.
    - `borderType` :
        - `cv2.BORDER_DEFAULT`
        - `cv2.BORDER_CONSTANT`
        - `cv2.BORDER_REPLICATE`
        - `cv2.BORDER_REFLECT`
        - `cv2.BORDER_WRAP`
        - `cv2.BORDER_ISOLATED`

____
### EXAMPLE 1 | Downscale Image

In [ ]:
# load image lena.jpg
img = cv2.imread("lena.jpg")

# get image shape
h, w, c = img.shape

# apply down scaling to one-quarter of the original image size (half width and half height)
img_PD = cv2.pyrDown(img, dstsize=(w//2, h//2))

# show result
cv2.imshow("pyramid downscale 1/2", img_PD)
cv2.imshow("original", img)
cv2.waitKey(0)
cv2.destroyAllWindows()

<br><br><br>
___
### EXAMPLE 2 | Upscale Image

In [ ]:
# load image lena.jpg
img = cv2.imread("lena.jpg")

# get image shape
h, w, c = img.shape

# apply up scaling to one-quarter of the original image size (half width and half height)
img_PU = cv2.pyrUp(img, dstsize=(w*2, h*2))

# show result
cv2.imshow("pyramid upscale 2x", img_PU)
cv2.imshow("original", img)
cv2.waitKey(0)
cv2.destroyAllWindows()

<br><br><br>
___
### EXAMPLE 3 | Downscale & Upscale Image by press key

In [ ]:
# load image lena.jpg
img = cv2.imread("lena.jpg")



# loop until user press 'q' key
while True :

    # get current image shape (in case the image is resized)
    h, w, c = img.shape

    # show image and wait for key press
    cv2.imshow("Result Window", img)
    key = cv2.waitKey(0)
    
    
    if key == ord("q"):

        break # exit loop if 'q' key is pressed

    elif key == ord('i'):

        # apply up scaling to one-quarter of the original image size (half width and half height)
        img = cv2.pyrUp(img, dstsize=(w*2, h*2))
        print ('Zoom In: Image x 2')
        
    elif key == ord('o'):

        # apply down scaling to one-quarter of the original image size (half width and half height)
        img = cv2.pyrDown(img, dstsize=(w//2, h//2))
        print ('Zoom Out: Image / 2')



# close all windows
cv2.destroyAllWindows()

<br><br><br>
___
# <font color="orange">1.2 Laplacian Pyramid</font>

- <font color="orange">Laplacian Pyramid</font> can be used for <font color="orange">edge detection</font>.
- $i$ layer on <font color="orange">Laplacian Pyramid</font> ($L_i$) created by <font color="orange">Gaussian Pyramid</font> $i$ ($G_i$) substracted by `cv2.pyrUp()` for Gaussian Pyramid layer in $i + 1$ (($G_{i+1}$)).<br><br>
$L_i = G_i - pyrUp(G_{i+1})$<br><br>
<img src="resource/Laplacian-Pyramids.png" style="width:400px"></img><br>
*Example of 3 stage Laplacian Pyramid* 

<br><br><br>
____

### EXAMPLE 4 | Apply Laplacian Pyramid (Single Stage)
<img src="resource/Laplacian-Pyramids-1.png" style="width:500px"></img>

In [ ]:
# load image lena.jpg as Gaussian Pyramid layer 0 (GP_0)
GP_0 = cv2.imread("lena.jpg") # 512x512

# create Gaussian Pyramid layer 1 (GP_1)
GP_1 = cv2.pyrDown(GP_0) # 256x256 --> downscale (gaussian pyramid)

# create Laplacian Pyramid layer 0 (LP_0) by subtracting the upscaled version of GP_1 from GP_0 using `cv2.subtract()`
LP_0 = cv2.subtract(GP_0, cv2.pyrUp(GP_1)) # 512x512 --> laplacian pyramid


# show result
cv2.imshow("GP 0", GP_0)
cv2.imshow("GP 1", GP_1)
cv2.imshow("LP 0", LP_0)
cv2.waitKey(0)
cv2.destroyAllWindows()

- Above example demonstrates how to apply <font color="orange">Laplacian Pyramid</font> for <font color="orange">edge detection</font>. The left image is the original image, while the right image is the result of applying <font color="orange">Laplacian Pyramid</font>.<br><br>
- Compare the result with <font color="orange">Canny Edge Detection</font>, which is a popular edge detection algorithm. 
- The result of <font color="orange">Laplacian Pyramid is more smooth</font> than <font color="orange">Canny Edge Detection</font>, which is <font color="orange">more sensitive to noise</font>. 
    - However, <font color="orange">Canny Edge Detection can detect more edges</font> than <font color="orange">Laplacian Pyramid</font>. 
    - The choice of edge detection algorithm depends on the application and the desired result.<br><br>
- `cv2.subtract()` is used to perform the subtraction between two images. 
    - The function takes two images as input and returns the difference between them.

<br><br><br>
___

### EXAMPLE 4 | Apply Laplacian Pyramid (3 Stage)
<img src="resource/Laplacian-Pyramids.png" style="width:500px"></img>

In [ ]:
# define empty list of all Gaussian Pyramid Image
GP_list = []

# load Original Image as Gaussian Pyramid layer 0 (GP_0)
GP = cv2.imread("lena.jpg")

# show gaussian pyramid level 0
cv2.imshow("GP Level 0 - Shape: " + str(GP.shape), GP)


# Insert Original Image as Base Gaussian Pyramid (512x512)
GP_list.append(GP) 

# loop to create Gaussian Pyramid layers 1 to 3 (GP_1 to GP_3)
for i in range (1, 4): # --> i : {1, 2, 3}

    # Apply Gaussian Pyramid & Append to list of Gaussian Pyramid
    GP = cv2.pyrDown(GP)
    GP_list.append(GP)

    # show image for each level of Gaussian Pyramid
    cv2.imshow("GP Level " + str(i) + " - Shape: " + str(GP.shape), GP)

# close all windows
cv2.waitKey(0)
cv2.destroyAllWindows()

- On above implementation, we have, <br><br>
    $GP_0$ : 512x512 (original image)<br>
    $GP_1$ : 256x256 <br>
    $GP_2$ : 128x128 <br>
    $GP_3$ : 64x64 <br><br>
- Next, try to calculate $LP_i$, where $LP_i = GP_i - pyrUp(GP_{i+1})$, <br><br>
    $LP_0$ : 64x64 (lowest GP image $GP_3$) <br>
    $LP_1$ : 128x128 ($GP_2 - pyrUp(GP_3)$) <br>
    $LP_2$ : 256x256 ($GP_1 - pyrUp(GP_2)$) <br>
    $LP_3$ : 512x512 ($GP_0 - pyrUp(GP_1)$) <br>

In [ ]:
# insert lower GP image (index 3) as the first layer of Laplacian Pyramid (LP_0)
LP = GP_list[3] # image size 64x64

# show LP Level 0
cv2.imshow("LP Level 0 - Shape: " + str(LP.shape), LP)

# loop to create Laplacian Pyramid layers 1 to 3 (LP_1 to LP_3) 
for i in range(2, -1, -1): # ---> i : {3, 2, 1}

    #  subtract the upscaled version of GP_i+1 from GP_i using `cv2.subtract()` to create LP_i
    LP =  cv2.subtract(GP_list[i], cv2.pyrUp(GP_list[i+1])) # GPi - pyrUp(GPi+1)

    # show current level of Laplacian Pyramid
    cv2.imshow("LP Level " + str(-1*i) + " - Shape: " + str(LP.shape), LP)

# close all windows
cv2.waitKey(0)
cv2.destroyAllWindows()

- Above example shows the result of applying Laplacian Pyramid that beneficial for edge detection. 

<br><br><br>

____
## EXAMPLE 5 : Image Pyramid for Image Stitching

- Simple Striching using Numpy slicing (just for comparion)

In [ ]:
# VERTICAL STITCHING (NUMPY)

# load image img1 & img2 (img1 & img2 have the same size)
img1 = cv2.imread("apple.jpg")
img2 = cv2.imread("orange.jpg")

# get first image shape
h, w, c = img1.shape


result = np.zeros_like(img1) # create black image with size & type similar to img1
result[:, :w//2] = img1[:, :w//2] # fill left side result matrix by left side img1
result[:, w//2:] = img2[:, w//2:] # fill right side result matrix by right side img2


# show result
cv2.imshow("stiching image", result)
cv2.waitKey(0)
cv2.destroyAllWindows()

In [ ]:
# HORIZONTAL STITCHING (NUMPY)

# load image img1 & img2
img1 = cv2.imread("apple.jpg")
img2 = cv2.imread("orange.jpg")

# get first image shape
h, w, c = img1.shape
print(h,w,c)

result = np.zeros_like(img1) # create black image with size & type similar to img1
result[:h//2, :] = img1[:h//2, :] # fill upper side result matrix by left side img1
result[h//2:, :] = img2[h//2:, :] # fill lower side result matrix by right side img2

# show result
cv2.imshow("stiching image", result)
cv2.waitKey(0)
cv2.destroyAllWindows()

- Above example is just for comparison, the result is not good because there is no <font color="orange">blending process</font>. The result is just a simple concatenation of two images.
- The <font color="orange">blending process</font> is important to create a <font color="orange">smooth transition</font> between two images, especially when the images have <font color="orange">different lighting conditions or colors</font>.
- Simple image stitching using numpy may not look good due to discontinuities between images.
<br><br><br>

#### <font color="orange">Simple Image Stitching using Image Pyramid</font>

In [ ]:
# VERTICAL STITCHING (IMAGE PYRAMID)

# load image img1 & img2 (img1 & img2 have the same size)
img1 = cv2.imread("apple.jpg")
img2 = cv2.imread("orange.jpg")


# Apply Gausian Pyramid to Image 1 and Image 2 and store into GP1_list & GP2_list

GP1 = img1.copy()
GP1_list = [GP1] 
for i in range (6): #--> i : {0, 1, 2, 3, 4, 5}
    GP1 = cv2.pyrDown(GP1) # downscale image by half
    GP1_list.append(GP1) # append current GP1 to list of GP1


GP2 = img2.copy()
GP2_list = [GP2] 
for i in range (6): #--> i : {0, 1, 2, 3, 4, 5}
    GP2 = cv2.pyrDown(GP2) # downscale image by half
    GP2_list.append(GP2) # append current GP2 to list of GP2



# Apply Laplacian Pyramid to Image 1 and Image 2 to generate Edge Image 
# Store into LP1_list & LP2_list

LP1_list = [GP1_list[5]]  # insert lower GP1 image (index 5) to LP1
for i in range (4, -1, -1): # ---> i : {4, 3, 2, 1, 0}
    LP1 =  cv2.subtract(GP1_list[i], cv2.pyrUp(GP1_list[i+1])) # GPi - pyrUp(GPi+1)
    LP1_list.append(LP1)
    

LP2_list = [GP2_list[5]] # insert lower GP2 image (index 5) to LP2
for i in range (4, -1, -1): # ---> i : {4, 3, 2, 1, 0}
    LP2=  cv2.subtract(GP2_list[i], cv2.pyrUp(GP2_list[i+1])) # GPi - pyrUp(GPi+1)
    LP2_list.append(LP2)




# Sticthing the laplacian image for all stage
# this process is to create a stiched image for each level of Laplacian Pyramid, 
# so we will have 6 stiched image (from level 0 to level 5)
LS = [] 
for L1, L2 in zip(LP1_list, LP2_list):
    h, w, c = L1.shape

    result = np.zeros_like(L1) # create black image with size & type similar to Laplacian Image 1
    result[:h//2, :] = L1[:h//2, :] # fill upper side result matrix by left side Laplacian Image 1
    result[h//2:, :] = L2[h//2:, :] # fill lower side result matrix by right side Laplacian Image 2
    LS.append(result)




# Add all stiched image into a single image called 'output'
# formula : output_i = pyrUp(output_i-1) + LSi
output = LS[0]
for i in range(1, 6):
    output = cv2.add(cv2.pyrUp(output), LS[i]) # output_i = pyrUp(output_i-1) + LSi
    

# show result
cv2.imshow("stiching image pyramid", output)
cv2.waitKey(0)
cv2.destroyAllWindows()

<br><br><br>
____

### EXAMPLE 6 : PERFORMANCE COMPARIOSN EDGE DETECTION 
- Canny Edge Detection vs Morphological Gradient vs Laplacian Pyramid

In [ ]:
# EDGE DETECTION -- MORPHOLOGICAL GRADIENT

times =[]
for i in range (100) : # run 100 times to get average execution time
    
    # get start time
    e1 = cv2.getTickCount()
    
    # ---------------------------------------------------------------------------------

    # load image
    img = cv2.imread('number_plate.jpg')

    # convert to gray
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

    # apply morphological gradient with kernel 3x3
    kernel = np.ones((3,3),np.uint8)
    gradient = cv2.morphologyEx(gray, cv2.MORPH_GRADIENT, kernel, iterations = 1)

    # apply simple thresholding TOZERO 
    ret, thresh = cv2.threshold(gradient, 0, 255, cv2.THRESH_TOZERO + cv2.THRESH_OTSU)

    # find contour & draw contour from binary image
    contours, hierarchy = cv2.findContours(thresh, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_NONE)

    for cnt in contours:
        cv2.drawContours(img, [cnt], -1, (0,0,255), 1)

    # ---------------------------------------------------------------------------------
    # calculate execution time
    e2 = cv2.getTickCount()
    t = (e2 - e1)/cv2.getTickFrequency()
    times.append(t)


print("Average execution : %.5f s" % np.array(times).mean())


# show image
cv2.imshow("Morphological Gradient", gradient)
cv2.imshow("Edge - Thresholding", thresh)
cv2.imshow("Original", img)
cv2.waitKey(0)
cv2.destroyAllWindows()

In [ ]:
# EDGE DETECTION -- CANNY EDGE DETECTION

times =[]
for i in range (100) : # run 100 times to get average execution time
    
    # get start time
    e1 = cv2.getTickCount()
    
    # ---------------------------------------------------------------------------------

    # load image
    img = cv2.imread('number_plate.jpg')

    # convert to gray
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

    # apply Canny Edge Detection
    canny = cv2.Canny(gray, 220, 230)

    # apply simple thresholding TOZERO 
    ret, thresh = cv2.threshold(canny, 0, 255, cv2.THRESH_TOZERO + cv2.THRESH_OTSU)

    # find contour & draw contour from binary image
    contours, hierarchy = cv2.findContours(thresh, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_NONE)

    for cnt in contours:
        cv2.drawContours(img, [cnt], -1, (0,0,255), 1)

    # ---------------------------------------------------------------------------------
    # calculate execution time
    e2 = cv2.getTickCount()
    t = (e2 - e1)/cv2.getTickFrequency()
    times.append(t)


print("Average execution : %.5f s" % np.array(times).mean())


# show image
cv2.imshow("Canny Edge Detection", canny)
cv2.imshow("Edge - Thresholding", thresh)
cv2.imshow("Original", img)
cv2.waitKey(0)
cv2.destroyAllWindows()

In [ ]:
# EDGE DETECTION -- LAPLACIAN PYRAMID
times =[]
for i in range (100) : # run 100 times to get average execution time
    
    # get start time
    e1 = cv2.getTickCount()
    # ---------------------------------------------------------------------------------


    # load image
    img = cv2.imread('number_plate.jpg')

    # convert to gray
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

    # apply Laplacian Pyramid
    GP0 = cv2.pyrDown(gray) # --> downscale (gaussian pyramid)
    #GP1 = cv2.pyrDown(GP0) # --> downscale (gaussian pyramid)
    LP0 = cv2.subtract(gray, cv2.pyrUp(GP0)) # --> laplacian pyramid
    #LP1 = cv2.subtract(gray, cv2.pyrUp(GP1)) # --> laplacian pyramid

    # apply simple thresholding TOZERO 
    ret, thresh = cv2.threshold(LP0, 0, 255, cv2.THRESH_TOZERO + cv2.THRESH_OTSU)

    # find contour & draw contour from binary image
    contours, hierarchy = cv2.findContours(thresh, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_NONE)

    for cnt in contours:
        cv2.drawContours(img, [cnt], -1, (0,0,255), 1)

    # ---------------------------------------------------------------------------------
    # calculate execution time
    e2 = cv2.getTickCount()
    t = (e2 - e1)/cv2.getTickFrequency()
    times.append(t)


print("Average execution : %.5f s" % np.array(times).mean())


# show image
cv2.imshow("Laplacian Pyramid", LP0)
cv2.imshow("Edge - Thresholding", thresh)
cv2.imshow("Original", img)
cv2.waitKey(0)
cv2.destroyAllWindows()

- Based on the result, we can conclude that <font color="orange">Laplacian Pyramid is slower</font> than Canny Edge Detection and Morphological Gradient. 
- However, the choice of edge detection algorithm depends on the application and the desired result.